# Propositional

```python
from logex import propositional as prop
```

## 1. Formula Instance

The main building block of propositional logics is a formula.

```python
prop.Formula(formula: str)
```

IMPORTANT: almost all functions of prop also work with simple str instances but it is not an optimal behaviour.

In [3]:
from logex import propositional as prop

formula = 'a & b -> (!c | d)'

formula = prop.Formula(formula)

formula

<Formula: a&b→(!c|d), vars: (a,b,c,d)>

Formula instance has certain attributes:
```python
.string
.original
.variables
.literals
.n_vars
```

IMPORTANT: there are many protected attributes, but they are not meant for access

In [4]:
print(formula.string)
print(formula.variables)
print(formula.literals)
print(formula.n_vars)
print(formula.original)

a&b→(!c|d)
['a', 'b', 'c', 'd']
['a', 'b', '!c', 'd']
4
a & b -> (!c | d)


General information about the Formula and its current state can be shown:
```python
.info()
```

In [5]:
print(formula.info())

Formula:            a&b→(!c|d)
Original input:     a & b -> (!c | d)
Variables:          a, b, c, d
Literals:           a, b, !c, d
N vars:             4
Subformulas:        Not yet computed, use .get_subformulas()
Type:               [ NOT CNF ] [ NOT DNF ] [ NOT AIG ] [ NOT NNF ]
NNF:                Not yet computed, use .to_nnf()
CNF:                Not yet computed, use .to_cnf()
DNF:                Not yet computed, use .to_dnf()
AIG:                Not yet computed, use .to_aig()
SAT?:               Not yet computed, use .is_sat()
UNSAT?:             Not yet computed, use .is_unsat()
VALID?:             Not yet computed, use .is_valid()
REFUTABLE?:         Not yet computed, use .is_refutable()
Model example:      Not yet computed, use .get_model()
Models so far:      Not yet computed, use .get_model()
Falsifying example: Not yet computed, use .get_falsifying()
Falsifying so far:  Not yet computed, use .get_falsifying()



Many other characteristics and conversions are computed with methods stated in the .info().

## 2. Structure

For some cases it might be important to collect subformulas of the formula:
```python
.get_subformulas() -> set[tuple[str, tuple]]
```

- each sub_formula consists of a tuple: (sub_formula, (left part, outer operation, right part))
- variables are not included in the set of subformulas but can be accessed through .variables

In [6]:
formula.get_subformulas()

{('!c', ('', '!', 'c')),
 ('!c|d', ('!c', '|', 'd')),
 ('a&b', ('a', '&', 'b')),
 ('a&b→(!c|d)', ('a&b', '→', '!c|d'))}

Also, it is possible to get the outermost opertaion of a formula
```python
prop.get_outermost_connective(formula | str) -> tuple
```

In [7]:
prop.get_outermost_connective(formula)

('→', ('a&b', '(!c|d)'))

The tree of operations:


In [8]:
prop.tree_of_operations(formula);

#################### TREE OF OPERATIONS ####################
Formula: a&b→(!c|d)
Tree:

                        (→)                          
              /‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾\                
            (&)                     (|)              
        /‾‾‾‾‾‾‾‾‾\             /‾‾‾‾‾‾‾‾‾\          
      (a)         (b)         (!)         (d)        
                              ‾‾\                    
                                (c)                  
############################################################


## 3. SAT checking

There are several method associated with SAT checking:
```python
.is_sat() -> bool
.is_unsat() -> bool
.is_valid() -> bool
.is_refutable() -> bool
.get_model() -> dict
.get_falsifying() -> dict
.get_truth_table() -> np.array, print
.get_all_models() -> set
.get_all_falsifying() -> set
```

Every time the method is called, its result and other logical consequences of this result are stored within Formula instance. When called again, the result will be taken from Formula itself to spare computation.

In [9]:
print(formula.is_sat())
print(formula.is_unsat())
print(formula.is_valid())
print(formula.is_refutable())

True
False
False
True


In [10]:
formula.get_truth_table();

####################### TRUTH TABLE ########################
Formula: a&b→(!c|d)
Truth Table:
		a b c d result
		0 0 0 0    1   
		0 0 0 1    1   
		0 0 1 0    1   
		0 0 1 1    1   
		0 1 0 0    1   
		0 1 0 1    1   
		0 1 1 0    1   
		0 1 1 1    1   
		1 0 0 0    1   
		1 0 0 1    1   
		1 0 1 0    1   
		1 0 1 1    1   
		1 1 0 0    1   
		1 1 0 1    1   
		1 1 1 0    0   
		1 1 1 1    1   
Valid? False
Satisfiable? True
Refutable? True
Unsatisfiable? False
############################################################


In [11]:
print('Model: ', formula.get_model())
print('Falsifying assignment: ', formula.get_falsifying())

Model:  {'a': 1, 'b': 1, 'c': 1, 'd': 1}
Falsifying assignment:  {'a': 1, 'b': 1, 'c': 1, 'd': 0}


Now we can check, how much the methods called before have changed the content of Formula instance:

In [12]:
print(formula.info())

Formula:            a&b→(!c|d)
Original input:     a & b -> (!c | d)
Variables:          a, b, c, d
Literals:           a, b, !c, d
N vars:             4
Subformulas:        4
Type:               [ NOT CNF ] [ NOT DNF ] [ NOT AIG ] [ NOT NNF ]
NNF:                !a|!b|(!c|d)
CNF:                !a|!b|!c|d
DNF:                Not yet computed, use .to_dnf()
AIG:                Not yet computed, use .to_aig()
SAT?:               True
UNSAT?:             False
VALID?:             False
REFUTABLE?:         True
Model example:      {'a': 1, 'b': 1, 'c': 1, 'd': 1}
Models so far:      15
Falsifying example: {'a': 1, 'b': 1, 'c': 1, 'd': 0}
Falsifying so far:  1



## 4. Conversions

<pre> ==> NNF (Negation Normal Form) </pre>
<pre> ==> CNF (Conjunction Normal Form) </pre>
(additionally has attributes .clauses and .clauses_qdimacs)
<pre> ==> DNF (Disjunction Normal Form) </pre>
(additionally has attribute .cubes)
<pre> ==> AIG (Add Inverter Graph) </pre>

In [13]:
nnf = formula.to_nnf()
print(nnf)

!a|!b|(!c|d)


In [14]:
cnf = formula.to_cnf()
print(cnf)
print(cnf.clauses)
print(cnf.clauses_qdmacs)

!a|!b|!c|d
{'!a|!b|!c|d'}
[[-1, -2, -3, 4]]


In [15]:
dnf = formula.to_dnf()
print(dnf)
print(dnf.cubes)

!b|d|!c|!a
['!b', 'd', '!c', '!a']


In [16]:
aig = formula.to_aig()
print(aig)

!(a&b&(c&!d))


## 5. Algorithms

### BCP (Binary Constraint Propagation)

Let's take example from graph coloring problem:

In [17]:
cnf = prop.graph_coloring(vertices=['a', 'b', 'c'], 
                          edges=[('a', 'b'), ('b', 'c'), ('c', 'a')],
                          num_colors=2)
cnf = prop.Formula(cnf, as_cnf=True)
print('\nCNF: ', cnf)

###################### GRAPH COLORING ######################
Vertices: ['a', 'b', 'c']
Edges: [('a', 'b'), ('b', 'c'), ('c', 'a')]
Number of colors: 2

Each vertex must have at least one color:
(a1|a2)&(b1|b2)&(c1|c2)

Each vertex must have at most one color:
(!a1|!a2)&(!b1|!b2)&(!c1|!c2)

Adjacent vertices cannot have the same color:
(!a1|!b1)&(!a2|!b2)&(!b1|!c1)&(!b2|!c2)&(!c1|!a1)&(!c2|!a2)

Result: (a1|a2)&(b1|b2)&(c1|c2)&(!a1|!a2)&(!b1|!b2)&(!c1|!c2)&(!a1|!b1)&(!a2|!b2)&(!b1|!c1)&(!b2|!c2)&(!c1|!a1)&(!c2|!a2)
############################################################

CNF:  (a1|a2)&(b1|b2)&(c1|c2)&(!a1|!a2)&(!b1|!b2)&(!c1|!c2)&(!a1|!b1)&(!a2|!b2)&(!b1|!c1)&(!b2|!c2)&(!c1|!a1)&(!c2|!a2)


In [18]:
bcp_1 = prop.bcp(cnf, 'a1')
bcp_2 = prop.bcp(bcp_1, 'c1')

########################### BCP ############################
Original CNF: (a1|a2)&(b1|b2)&(c1|c2)&(!a1|!a2)&(!b1|!b2)&(!c1|!c2)&(!a1|!b1)&(!a2|!b2)&(!b1|!c1)&(!b2|!c2)&(!c1|!a1)&(!c2|!a2)
BCP on literal a1
(!b1|!b2) ==> (!b1|!b2)
(!a1|!b1) ==> (!b1)
(!b2|!c2) ==> (!b2|!c2)
(!c1|!c2) ==> (!c1|!c2)
(!a2|!b2) ==> (!a2|!b2)
(b1|b2) ==> (b1|b2)
(!a1|!a2) ==> (!a2)
(c1|c2) ==> (c1|c2)
(!b1|!c1) ==> (!b1|!c1)
(!c1|!a1) ==> (!c1)
(a1|a2) ==> ⊤
(!c2|!a2) ==> (!a2|!c2)

Result: (!b1|!b2)&(!c1|!c2)&(!b2|!c2)&(b1|b2)&(!b1)&(!a2|!c2)&(c1|c2)&(!b1|!c1)&(!a2|!b2)&(!c1)&(!a2)
############################################################
########################### BCP ############################
Original CNF: (!b1|!b2)&(!c1|!c2)&(!b2|!c2)&(b1|b2)&!b1&(!a2|!c2)&(c1|c2)&(!b1|!c1)&(!a2|!b2)&!c1&!a2
BCP on literal c1
(!b1|!b2) ==> (!b1|!b2)
(!c1|!c2) ==> (!c2)
(!b2|!c2) ==> (!b2|!c2)
(b1|b2) ==> (b1|b2)
(!b1) ==> (!b1)
(!a2|!c2) ==> (!a2|!c2)
(c1|c2) ==> ⊤
(!b1|!c1) ==> (!b1)
(!a2|!b2) ==> (!a2|!b2)
(!c1

### DPLL

In [19]:
prop.dpll(cnf)

########################### DPLL ###########################

Formula: (a1|a2)&(b1|b2)&(c1|c2)&(!a1|!a2)&(!b1|!b2)&(!c1|!c2)&(!a1|!b1)&(!a2|!b2)&(!b1|!c1)&(!b2|!c2)&(!c1|!a1)&(!c2|!a2)
BCP on "a1":	(!b1|!b2)&(!c1|!c2)&(!b2|!c2)&(b1|b2)&!b1&(!a2|!c2)&(c1|c2)&(!b1|!c1)&(!a2|!b2)&!c1&!a2
BCP on "b1":		⊥
BCP on "!b1":		(!b2|!c2)&(!c1|!c2)&(!a2|!c2)&(c1|c2)&(!a2|!b2)&b2&!c1&!a2
BCP on "c2":			!b2&(!a2|!b2)&b2&!c1&!a2
BCP on "b2":				⊥
BCP on "!b2":				⊥
BCP on "!c2":			c1&(!a2|!b2)&b2&!c1&!a2
BCP on "a2":				⊥
BCP on "!a2":				b2&!c1&c1
BCP on "c1":					⊥
BCP on "!c1":					⊥
BCP on "!a1":	a2&(!b1|!b2)&(!c1|!c2)&(!b2|!c2)&(b1|b2)&(!a2|!c2)&(c1|c2)&(!b1|!c1)&(!a2|!b2)
BCP on "b2":		a2&(!c1|!c2)&(!a2|!c2)&!b1&(c1|c2)&(!b1|!c1)&!c2&!a2
BCP on "c2":			⊥
BCP on "!c2":			a2&c1&!b1&(!b1|!c1)&!a2
BCP on "a2":				⊥
BCP on "!a2":				⊥
BCP on "!b2":		a2&(!c1|!c2)&(!a2|!c2)&(c1|c2)&(!b1|!c1)&b1
BCP on "c1":			a2&(!a2|!c2)&!b1&b1&!c2
BCP on "a2":				!c2&!b1&b1
BCP on "b1":					⊥
BCP on "!b1":					⊥
BCP o

'UNSAT'

### Resolution

There are several function for resolution:

```python
possible_pivots(formula: Union[str, Formula], 
                formula_1: Union[str, Formula] = None) -> list[str]
```

If one CNF is provided: returns all possible pivots to apply to this CNF
If 2 formulas provided: assums that each is a clause and returns possible pivots.


```python
binary_resolution(clause_1: Union[str, Formula], 
                  clause_2: Union[str, Formula], 
                  pivot: str = None,
                  allow_tautology: bool = True,
                  return_all: bool = False) -> str | bool
```
Resolves 2 clauses. If impossible - returns False.

- pivot: if pivot is given - tries to resolve on pivot, else - tries every possible pivot.
- allow_tautology: allows resolvent(s) to be tautology, else converts to constant(s)
- return_all: allows to return all possible results if pivot wa not provided


```python
resolution(formula: Union[str, Formula], 
           formula_2: Union[str, Formula] = None, 
           pivot: str = None,
           allow_tautology: bool = True,
           return_all: bool = False) -> Any
```

If a single formula is provided - perform resolution within CNF. Else - treats both formulas as clauses.

- pivot: if pivot is given - tries to resolve on pivot, else - tries every possible pivot.
- allow_tautology: allows resolvent(s) to be tautology, else converts to constant(s)
- return_all: allows to return all possible results instead of one at maximum.

```python
resolution_refutation(formula: Union[str, Formula]) -> print
```

Tries to construct resolution refutation of UNSAT CNF.



In [20]:
prop.possible_pivots(
    '(!a | b | !c) & (d | !b | a)'
)

['a', 'b']

In [21]:
prop.binary_resolution(
    '(!a | b | !c)', 
    '(d | !b | a)'
)
prop.binary_resolution(
    '(!a | b | !c)', 
    '(d | !b | a)',
    return_all=True
)
prop.binary_resolution(
    '(!a | b | !c)', 
    '(d | !b | a)',
    return_all=True,
    allow_tautology=False
);

######################## RESOLUTION ########################
[ single output ] [ tautology allowed ]
Clause 1: (!a | b | !c)
Clause 2: (d | !b | a)
Possible pivots are: ['a', 'b']
RES(Clause 1, Clause 2; on "a") = !b|!c|d|b
############################################################
######################## RESOLUTION ########################
[ multiple output ] [ tautology allowed ]
Clause 1: (!a | b | !c)
Clause 2: (d | !b | a)
Possible pivots are: ['a', 'b']
RES(Clause 1, Clause 2; on "a") = !b|!c|d|b
RES(Clause 1, Clause 2; on "b") = !c|d|a|!a
############################################################
######################## RESOLUTION ########################
[ multiple output ] [ tautology not allowed ]
Clause 1: (!a | b | !c)
Clause 2: (d | !b | a)
Possible pivots are: ['a', 'b']
RES(Clause 1, Clause 2; on "a") = ⊤
RES(Clause 1, Clause 2; on "b") = ⊤
############################################################


In [22]:
prop.resolution(
    '(!a | b | !c) & (d | !b | a) & (e | !d) & (e | f)'
    )
prop.resolution(
    '(!a | b | !c) & (d | !b | a) & (e | !d) & (e | f)',
    return_all=True
    )
prop.resolution(
    '(!a | b | !c) & (d | !b | a) & (e | !d) & (e | f)',
    return_all=True,
    allow_tautology=False
    );

######################## RESOLUTION ########################
[ single output ] [ tautology allowed ]
CNF: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)
Pivot is not provided
Possible pivots are: ['a', 'b', 'd']
Pivot "a" generates the following resolvents:
	RES((d|!b|a), (!a|b|!c), on "a") = !b|d|!c|b
############################################################

######################## RESOLUTION ########################
[ multiple output ] [ tautology allowed ]
CNF: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)
Pivot is not provided
Possible pivots are: ['a', 'b', 'd']
Pivot "a" generates the following resolvents:
	RES((d|!b|a), (!a|b|!c), on "a") = !b|d|!c|b
Pivot "b" generates the following resolvents:
	RES((d|!b|a), (!a|b|!c), on "b") = !c|d|a|!a
Pivot "d" generates the following resolvents:
	RES((e|!d), (d|!b|a), on "d") = !b|a|e
############################################################

######################## RESOLUTION ########################
[ multiple output ] [ tautology not allowed ]
CNF: (!a|b|

In [23]:
prop.resolution_refutation(cnf)

54 trial resolutions were performed
Found at depth 4


                           [!a1|!b1]       [a1|a2]        [!b2|!c2]        [c1|c2]        [!c1|!a1]      [a1|a2]                           [b1|b2]       [!b1|!c1]      [c1|c2]        [!c2|!a2]     
                                   \_______/                      \________/                      \______/                                       \_______/                    \________/             
            [b1|b2]               a2|!b1                              !b2|c1                    a2|!c1                                               b2|!c1                   !a2|c1                
                  \_______________/                                        \____________________/                                                         \___________________/                     
                      a2|b2                                                      !b2|a2                                  [!a2|!b2]                         

### Blocked Clauses

Several functions are availbale for work with blocked clauses:


```python
is_blocked_on_literal(clause: Union[str, Formula], 
                      in_formula: Union[str, Formula], 
                      literal: str) -> bool
```

Checks if clause is blocked in CNF on the given literal.

```python
is_blocked(clause: Union[str, Formula], 
           in_formula: Union[str, Formula]) -> list[str]
```

Checks if clause is blocked in CNF on any literal.

```python
blocked_clauses(formula: Union[str, Formula]) -> dict:
```

Searches for blocked clauses in CNF and returned a dict of blocked clauses with kays = clause, value = blocking literals.

```python
blocked_clauses_elimination(formula: Union[str, Formula]) -> print
```

Perform BCE until fixed point.

In [24]:
cnf = prop.Formula('(!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)', as_cnf=True)

prop.is_blocked_on_literal(clause = '(!a|b|!c)', 
                           in_formula = cnf,
                           literal = '!a');
prop.is_blocked(clause = '(d|!b|a)', 
                in_formula = cnf);


###################### BLOCKED CLAUSE ######################
Checking if clause: !a|b|!c
is blocked in formula: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)
On literal "!a"

Possible RES with clause (d|!b|a) = (!b|d|!c|b) (tautology)

Result: BLOCKED
############################################################

###################### BLOCKED CLAUSE ######################
Checking if clause: d|!b|a
is blocked in formula: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)

On literal a: True
On literal !b: True
On literal d: False

Result: BLOCKED on literals ['a', '!b']
############################################################



In [25]:
prop.blocked_clauses(cnf);

###################### BLOCKED CLAUSE ######################
Searching for blocked clauses in: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)

Clause e|!d is blocked on literals ['e']
Clause d|!b|a is blocked on literals ['a', '!b']
Clause e|f is blocked on literals ['e', 'f']
Clause !a|b|!c is blocked on literals ['!a', 'b', '!c']

Result 4/4 clauses are blocked
############################################################



In [26]:
prop.blocked_clauses_elimination(cnf);

########################### BCE ############################
Step 1)
	Current residual: (!a|b|!c)&(d|!b|a)&(e|!d)&(e|f)
	Blocked clauses: {'e|!d': ['e'], 'd|!b|a': ['a', '!b'], 'e|f': ['e', 'f'], '!a|b|!c': ['!a', 'b', '!c']}
	Eliminating: e|!d

Step 2)
	Current residual: (d|!b|a)&(e|f)&(!a|b|!c)
	Blocked clauses: {'d|!b|a': ['a', '!b', 'd'], 'e|f': ['e', 'f'], '!a|b|!c': ['!a', 'b', '!c']}
	Eliminating: d|!b|a

Step 3)
	Current residual: (e|f)&(!a|b|!c)
	Blocked clauses: {'e|f': ['e', 'f'], '!a|b|!c': ['!a', 'b', '!c']}
	Eliminating: e|f

Step 4)
	Current residual: !a|b|!c
	Blocked clauses: {'!a|b|!c': ['!a', 'b', '!c']}
	Eliminating: !a|b|!c

Step 5)
	Current residual: ⊤
	Blocked clauses: {'⊤': False}
Result: ⊤
############################################################

